In [ ]:
seed = 0

wav_file = (
    "/home/marnix/thesis/data/OPENGLOT/RepositoryI/Vowel_I/I_breathy_280Hz.wav"
)

In [3]:
print(seed)

0


In [ ]:
print(wav_file)

In [5]:
import scrapbook as sb

sb.glue("seed", seed)
sb.glue("wav_file", wav_file)